In [1]:
import furuta_sympy_calc as furu_calc
import pandas as pd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import furuta_params

params = furuta_params.FurutaParams()
params.mu_H1 = 0.0
params.mu_H2 = 0.0
tau_max = params.tau_max

a) Berechnen Sie die Systemmatrix A und den Eingangsvektor b mit den in Tab. 1
gegebenen Parametern.


In [2]:
#Arbeitspunkt 0, 0, 0, 0, 0
A_lin, b_lin = furu_calc.get_linear_furuta(params, 0, 0, 0, 0, 0)
A_lin

Matrix([
[0,                 1,                 0,                    0],
[0, -17.5161709980497,  1.07509937893889, 0.000443968358900185],
[0,                 0,                 0,                    1],
[0,  16.9687906543607, -62.3540025233471,   -0.025749437413378]])

In [3]:
b_lin

Matrix([
[                0],
[  34.406150064918],
[                0],
[-33.3309578753893]])

b) Überprüfen Sie die Steuerbarkeit des linearisierten Modells.

In [4]:
# SymPy -> NumPy
A_np = np.array(A_lin.tolist(), dtype=float)
b_np = np.array(b_lin.tolist(), dtype=float)

n = A_np.shape[0]

# Steuerbarkeitsmatrix S = [B, AB, A^2B, ...]
S_blocks = [np.linalg.matrix_power(A_np, k) @ b_np for k in range(n)] #liste von spalten vektoren
S_np = np.hstack(S_blocks) #spaletenvektoren werden zu matrix zusammen gefügt

rankS = np.linalg.matrix_rank(S_np)  # nutzt intern Toleranz (SVD-basiert)

print("S shape:", S_np.shape)
print("rank(S):", rankS)
print("voll steuerbar?", rankS == n)

S shape: (4, 4)
rank(S): 4
voll steuerbar? True


c) Berechnen Sie die Eigenwerte der Systemmatrix.

In [5]:
np.linalg.eig(A_np)

EigResult(eigenvalues=array([  0.        +0.j        , -17.46681993+0.j        ,
        -0.03755025+7.84119401j,  -0.03755025-7.84119401j]), eigenvectors=array([[ 1.00000000e+00+0.j        , -4.44670392e-02+0.j        ,
        -8.23695775e-04+0.00037581j, -8.23695775e-04-0.00037581j],
       [ 0.00000000e+00+0.j        ,  7.76697767e-01+0.j        ,
        -2.91590107e-03-0.00647287j, -2.91590107e-03+0.00647287j],
       [ 0.00000000e+00+0.j        ,  3.59123592e-02+0.j        ,
        -6.05792834e-04-0.12650086j, -6.05792834e-04+0.12650086j],
       [ 0.00000000e+00+0.j        , -6.27274712e-01+0.j        ,
         9.91940495e-01+0.j        ,  9.91940495e-01-0.j        ]]))

d) Überführen Sie das linearisierte System in Regelungsnormalform (RNF) und führen 
Sie einen flachen Ausgang ein.


In [6]:
S_inv = np.linalg.inv(S_np)
q1T = S_inv[-1, :].reshape(1, -1) #reshape to explicitly make it a row vector

In [7]:
Q_blocks = [q1T @ np.linalg.matrix_power(A_np, k)for k in range(n)] #liste von zeilen vektoren
Q = np.vstack(Q_blocks)
Q_inv = np.linalg.inv(Q)
tol = 1e-13   # wählbar, z.B. 1e-12, 1e-9, je nach Skalierung

A_RNF = Q @ A_np @ Q_inv
A_RNF[np.abs(A_RNF) < tol] = 0.0
A_RNF

array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 0.00000000e+00, -1.07396023e+03, -6.27975005e+01,
        -1.75419204e+01]])

In [8]:
b_RNF = Q @ b_np
b_RNF[np.abs(b_RNF) < tol] = 0.0
b_RNF


array([[0.],
       [0.],
       [0.],
       [1.]])

e) Erarbeiten Sie auf Basis der RNF eine Steuerung, die zur Überführung zwischen 
stationären Regimen innerhalb eines vorgegebenes Zeitintervall geeignet ist.